In [25]:
import numpy as np
import xgboost as xgb
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [26]:
df_data=pd.read_csv('train.csv',encoding='utf-8')
df_label=pd.read_csv('train_target.csv',encoding='utf-8')
df_test=pd.read_csv('test.csv',encoding='utf-8')

In [27]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values
    summary['Uniques'] = df.nunique().values
    summary['Value Example'] = df.loc[0].values
    
    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2)
    return summary

In [28]:
X = df_data.drop(['id'], axis=1)
Y = df_label['target']
Z = df_test.drop(['id'], axis=1)

In [29]:
X['bank_pre'] = X['bankCard'][:-1]
X['x_277168'] = X['x_27'] * X['x_68'] * X['x_71'] 
X['certValid'] = X['lmt'] * X['basicLevel']


Z['bank_pre'] = Z['bankCard'][:-1]
Z['x_277168'] = Z['x_27'] * Z['x_68'] * Z['x_71']
Z['certValid'] = Z['lmt'] * Z['basicLevel']


In [31]:
try:
    X['temp'] =X['certValidStop']-X['certValidBegin']
except Exception as e:
    X['temp'] ='9999999999'
#X['cert'] = X['certId'].map(str)+X['temp'].map(str)
X['cert'] = X['temp'].astype(float)

In [32]:
try:
    Z['temp'] =Z['certValidStop']-Z['certValidBegin']
except Exception as e:
    Z['temp'] ='9999999999'
#Z['cert'] = Z['certId'].map(str)+Z['temp'].map(str)
Z['cert'] = Z['temp'].astype(float)

In [33]:
X = X.drop(['temp'], axis=1)
Z = Z.drop(['temp'], axis=1)
resumetable(X)

Dataset Shape: (132029, 107)


,Name,dtypes,Missing,Uniques,Value Example,Entropy
0,certId,int64,0,4033,6.405820e+05,11.10
1,loanProduct,int64,0,3,1.000000e+00,1.47
2,gender,int64,0,2,2.000000e+00,0.80
3,age,int64,0,38,2.700000e+01,4.64
4,dist,int64,0,3738,6.405000e+05,10.28
5,edu,int64,0,9,0.000000e+00,0.16
6,job,int64,0,13,8.000000e+00,2.27
7,lmt,float64,0,1718,5.963000e+00,8.74
8,basicLevel,int64,0,6,3.000000e+00,1.68
9,x_0,int64,0,2,0.000000e+00,0.02


In [34]:
#X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=.2,random_state=0)
X_train, Y_train = X, Y

In [35]:
# kf = KFold(n_splits=5, shuffle=False)
# flag =0 
# for train_index, test_index in kf.split(X):
#     X_train = X.loc[train_index]
#     Y_train = Y.loc[train_index]
#     X_test = X.loc[test_index]
#     Y_test = Y.loc[test_index]
#     model = xgb.XGBClassifier(base_score = 0.06)
#     model.fit(X_train,Y_train)
#     y_pred = model.predict_proba(Z)[:,1]
#     print(y_pred)
#     print(flag)
#     if flag == 0:
#         preds = np.array(y_pred)
#         flag = 1
#     else:
#         #sums =  np.row_stack((preds, y_pred)) 
#         sums = np.maximum(preds,y_pred)
#         preds =  sums
#         print(preds)

In [36]:
# preds.shape
# test_pred = preds
# test_pred

In [37]:
model = xgb.XGBClassifier(base_score = 0.06)
model.fit(X_train,Y_train)

XGBClassifier(base_score=0.06, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [38]:
importances = model.feature_importances_ 
indices = np.argsort(importances)[::-1]
feat_labels = X_train.columns
print("Feature ranking:") 
#    l1,l2,l3,l4 = [],[],[],[]
for f in range(X_train.shape[1]):
    print("%d. feature no:%d feature name:%s (%f)" % (f + 1, indices[f], feat_labels[indices[f]], importances[indices[f]]))
print (">>>>>", importances)

Feature ranking:
1. feature no:54 feature name:x_45 (0.070059)
2. feature no:55 feature name:x_46 (0.044027)
3. feature no:71 feature name:x_62 (0.038892)
4. feature no:1 feature name:loanProduct (0.038735)
5. feature no:103 feature name:bank_pre (0.033312)
6. feature no:3 feature name:age (0.027512)
7. feature no:7 feature name:lmt (0.025854)
8. feature no:105 feature name:certValid (0.025663)
9. feature no:90 feature name:bankCard (0.024821)
10. feature no:61 feature name:x_52 (0.024507)
11. feature no:6 feature name:job (0.023906)
12. feature no:77 feature name:x_68 (0.023725)
13. feature no:36 feature name:x_27 (0.023562)
14. feature no:8 feature name:basicLevel (0.023190)
15. feature no:37 feature name:x_28 (0.023051)
16. feature no:93 feature name:highestEdu (0.022962)
17. feature no:104 feature name:x_277168 (0.022059)
18. feature no:80 feature name:x_71 (0.021389)
19. feature no:81 feature name:x_72 (0.020828)
20. feature no:25 feature name:x_16 (0.020737)
21. feature no:84 fea

In [39]:
y_pred = model.predict_proba(X_train)[:,1]
test_pred = model.predict_proba(Z)[:,1]
# predictions = [round(value) for value in test_pred]
test_pred

array([0.00776364, 0.01350979, 0.00642696, ..., 0.00737331, 0.00546772,
       0.00337754], dtype=float32)

In [40]:
auc = roc_auc_score(Y_train.values, y_pred)
print(auc)

0.7954368961080982


In [41]:
sub = pd.read_csv('submission.csv',encoding='utf-8')
p = np.array(test_pred).reshape(-1,1)
sub['target'] = p

In [42]:
sub[['id','target']].to_csv('sub.csv',index=False)

线下AUC随base_score变化：
1      0.77
0.07 0.7872
0.06 0.7908
0.05 0.7897

delete 
0.7908
non-delete
0.7934
non-certid
0.7954